In [21]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [22]:
house_dict = []

for page in range(16, 21):
    linked = 'https://www.zillow.com/ca/duplex/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-130.93471906436687,"east":-107.64370343936687,"south":31.980790552461904,"north":48.0418909706512},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false},"isCondo":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:58.0) Gecko/20100101 Firefox/58.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[0].text
            except:
                zestimate = 'null'
            try:
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[1].text
                
            except:
                rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

https://www.zillow.com/homedetails/2230-Rutgers-Dr-Costa-Mesa-CA-92627/25460901_zpid/
zestimate $1,295,505
rent zestimate $2,700/mo
https://www.zillow.com/homedetails/2663-Nevada-Ave-Oroville-CA-95966/18226092_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/5313-Gordon-Dr-Sacramento-CA-95824/25819746_zpid/
zestimate $388,679
rent zestimate $1,475/mo
https://www.zillow.com/homedetails/8311-Country-Club-Ln-Auburn-CA-95602/17664504_zpid/
zestimate $1,109,565
rent zestimate $2,500/mo
https://www.zillow.com/homedetails/1402-Earl-St-Calistoga-CA-94515/2080131014_zpid/
zestimate $899,006
rent zestimate null
https://www.zillow.com/homedetails/601-Lincoln-Ave-Bakersfield-CA-93308/18921460_zpid/
zestimate $299,798
rent zestimate $1,200/mo
https://www.zillow.com/homedetails/9900-Myrtle-Ave-Lamont-CA-93241/18953413_zpid/
zestimate $285,119
rent zestimate $1,200/mo
https://www.zillow.com/homedetails/148-14th-St-Seal-Beach-CA-90740/25307702_zpid/
zestimate $1,848,280
rent

https://www.zillow.com/homedetails/200-W-Hawkeye-Ave-Turlock-CA-95380/2072078678_zpid/
zestimate $626,603
rent zestimate null
https://www.zillow.com/homedetails/214-Bancroft-Ave-San-Leandro-CA-94577/24876234_zpid/
zestimate $982,399
rent zestimate $2,750/mo
https://www.zillow.com/homedetails/7420-Alexander-Ct-Fair-Oaks-CA-95628/54916483_zpid/
zestimate $3,153,454
rent zestimate $1,850/mo
https://www.zillow.com/homedetails/577-W-Escalon-Ave-Clovis-CA-93612/82853774_zpid/
zestimate $730,156
rent zestimate $1,095/mo
https://www.zillow.com/homedetails/3296-Lime-St-Riverside-CA-92501/2071774288_zpid/
zestimate $730,506
rent zestimate null
https://www.zillow.com/homedetails/3291-Saint-Helena-Hwy-N-Saint-Helena-CA-94574/2101114166_zpid/
zestimate $11,775,642
rent zestimate $46,872/mo
https://www.zillow.com/homedetails/21115-7-Gardena-Dr-Cupertino-CA-95014/2072539472_zpid/
zestimate $2,144,797
rent zestimate null
https://www.zillow.com/homedetails/1241-Ocean-Front-Laguna-Beach-CA-92651/2555707

zestimate null
rent zestimate null
https://www.zillow.com/homedetails/105-Cypress-Ave-Vallejo-CA-94590/2071733278_zpid/
zestimate $816,573
rent zestimate null
https://www.zillow.com/homedetails/3601-Saviers-Rd-Oxnard-CA-93033/16396073_zpid/
zestimate $934,995
rent zestimate $2,450/mo
https://www.zillow.com/homedetails/1197-N-Normandie-Ave-Los-Angeles-CA-90029/20790381_zpid/
zestimate $778,470
rent zestimate $2,650/mo
https://www.zillow.com/homedetails/19-Loyola-Ter-San-Francisco-CA-94117/2071784491_zpid/
zestimate $1,889,453
rent zestimate null
https://www.zillow.com/homedetails/1838-8th-Ave-Olivehurst-CA-95961/2074548300_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/345-N-Van-Ness-Ave-Fresno-CA-93701/2072186072_zpid/
zestimate $293,563
rent zestimate null
https://www.zillow.com/homedetails/1245-Carmichael-Rd-Clio-CA-96106/112429817_zpid/
zestimate $8,000/mo
rent zestimate null
https://www.zillow.com/homedetails/2318-Olive-Ave-Altadena-CA-91001/20908497_zp

zestimate $621,476
rent zestimate $2,300/mo
https://www.zillow.com/homedetails/717-El-Tejon-Ave-Bakersfield-CA-93308/18920976_zpid/
zestimate $239,294
rent zestimate $1,200/mo
https://www.zillow.com/homedetails/1301-Turrill-Ave-San-Bernardino-CA-92411/2104928176_zpid/
zestimate $2,400/mo
rent zestimate null
https://www.zillow.com/homedetails/180-Hamilton-Ave-Rio-Vista-CA-94571/2072762507_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/6301-Newell-St-Huntington-Park-CA-90255/21072743_zpid/
zestimate $765,006
rent zestimate $2,600/mo
https://www.zillow.com/homedetails/42900-Mather-Ln-Laytonville-CA-95454/250119115_zpid/
zestimate $2,082,347
rent zestimate $2,100/mo
https://www.zillow.com/homedetails/815-Pacific-Ave-APT-1-Long-Beach-CA-90813/21235991_zpid/
zestimate $1,550/mo
rent zestimate null
https://www.zillow.com/homedetails/4552-4554-47th-St-San-Diego-CA-92115/2071625882_zpid/
zestimate $1,097,622
rent zestimate null
https://www.zillow.com/homedetails/736

In [23]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,TXR HOMES,"$1,295,000","2230 Rutgers Dr, Costa Mesa, CA 92627",2230 Rutgers Dr,CostaMesa,CA,92627,--,--,"2,490",null,null,MultiFamily,1958,Garage,null,"$1,295,505","$2,700/mo",$520,https://www.zillow.com/homedetails/2230-Rutger...
1,GOLDTOWN REALTY,"$235,000","2663 Nevada Ave, Oroville, CA 95966",2663 Nevada Ave,Oroville,CA,95966,3,2,"1,474",null,null,MultiFamily,1953,No Data,null,null,null,$159,https://www.zillow.com/homedetails/2663-Nevada...
2,,"$369,000","5313 Gordon Dr, Sacramento, CA 95824",5313 Gordon Dr,Sacramento,CA,95824,2,--,"1,410",null,null,MultiFamily,1949,"Garage - Detached,",null,"$388,679","$1,475/mo",$262,https://www.zillow.com/homedetails/5313-Gordon...
3,,"$1,100,000","8311 Country Club Ln, Auburn, CA 95602",8311 Country Club Ln,Auburn,CA,95602,4,4,"3,300",null,null,MultiFamily,1985,"Carport, Garage - Attached,",$41 monthly,"$1,109,565","$2,500/mo",$333,https://www.zillow.com/homedetails/8311-Countr...
4,,"$899,000","1402 Earl St, Calistoga, CA 94515",1402 Earl St,Calistoga,CA,94515,--,--,--,null,null,MultiFamily,1920,Uncovered,null,"$899,006",null,null,https://www.zillow.com/homedetails/1402-Earl-S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,,"$2,100,000","42900 Mather Ln, Laytonville, CA 95454",42900 Mather Ln,Laytonville,CA,95454,3,2,"2,544",null,null,MultiFamily,1950,"Unpaved, Gravel",null,"$2,082,347","$2,100/mo",$825,https://www.zillow.com/homedetails/42900-Mathe...
196,T.N.G. REAL ESTATE CONSULTANTS,"$224,000","815 Pacific Ave APT 1, Long Beach, CA 90813",815 Pacific Ave APT 1,LongBeach,CA,90813,1,1,569,null,null,MultiFamily,1955,None,$160 monthly,"$1,550/mo",null,$394,https://www.zillow.com/homedetails/815-Pacific...
197,,"$1,049,900","4552-4554 47th St, San Diego, CA 92115",4552-4554 47th St,SanDiego,CA,92115,3,3,--,null,null,MultiFamily,1942,0 spaces,null,"$1,097,622",null,null,https://www.zillow.com/homedetails/4552-4554-4...
198,CINDY MARK REALTY,"$329,000","73648 Buena Vista Dr, Twentynine Palms, CA 92277",73648 Buena Vista Dr,TwentyninePalms,CA,92277,--,--,"2,000",null,null,MultiFamily,1954,Driveway,null,"$308,914",$900/mo,$165,https://www.zillow.com/homedetails/73648-Buena...


In [24]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2021_Multi_Family_Data_page16-20.csv", index=False, header=True)